In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/clblp-task-2/sample_submission.csv
/kaggle/input/clblp-task-2/train.csv
/kaggle/input/clblp-task-2/test.csv


In [2]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Dense, LSTM, Bidirectional
from keras.models import Sequential
from keras import regularizers
import tensorflow as tf
import tensorflow_addons as tfa
from keras import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

# **Read Train and Test Data**

In [3]:
train_data = pd.read_csv(os.path.join(dirname, 'train.csv'), encoding = 'UTF-8')
train_data

,Reviews,Sentiment
0,মুরগির chaap সত্যিই মুখরোচক হয়। আমি এটা পছন্দ...,1
1,একটি ম্যাসেজ থেরাপিস্ট যারা ব্যাখ্যা করেছেন যে...,0
2,এটির মান একটি যুগ্ম মান নয়।,0
3,আমি মনে করি তারা একটি উন্নততর প্রধান ভাড়া প্র...,0
4,প্রস্তুতিটা খুবই ভালো - সত্যিই সুস্বাদু। যত তা...,1
...,...,...
6854,ইস্ আমি আমার টাকা নষ্ট না করেছে! আমি সাধারণত ক...,1
6855,আমি মনে করি শেফ Silvana মধ্যে Esparza প্রথম রে...,0
6856,অস্বাস্থ্যকর অস্বাস্থ্যকর খাদ্য সন্ত্রাসী,1
6857,খাদ্য প্রতিটি penny.ব্যবহারপরিবার ও বন্ধুদের স...,1


In [4]:
test_data = pd.read_csv(os.path.join(dirname, 'test.csv'), encoding = 'UTF-8')
test_data

,Id,Reviews
0,0,"সুতরাং, গতকাল সকালে, আমি আসলে একটি prego ক্ষুধ..."
1,1,এটা আমাদের জন্য বিশেষ কিছু নেই। আমরা ইফতার ও ব...
2,2,"ঢাকায় ভালো ঠিকানা, বিশেষ করে যদি আপনি ওয়েস্ট..."
3,3,আমার জন্য তা গন্ধ আমি সবসময় পছন্দ চাকন মত ছিল...
4,4,খাবার ও পরিবেশ এত ভাল! সঙ্গীর সঙ্গে থাকা ভালো ...
...,...,...
1710,1710,আমি খাদ্য :) খুব সুস্বাদু এবং রেস্টুরেন্ট এর প...
1711,1711,খারাপ আচরণ। খারাপ ম্যানেজমেন্ট সিস্টেম। খারাপ ...
1712,1712,নিশ্চিতভাবে অধিকারী না $ 3 আমি দেওয়া।
1713,1713,আলু চিপ অর্ডার দু: খিত ছিল ... আমি সম্ভবত গণনা...


In [5]:
print("Total Reviews:", len(train_data))
print("Total Positive Reviews:", len(train_data[train_data['Sentiment'] == 1]))
print("Total Negative Reviews:", len(train_data[train_data['Sentiment'] == 0]))

Total Reviews: 6859
Total Positive Reviews: 3989
Total Negative Reviews: 2870


In [6]:
print("Total Test Reviews:", len(test_data))

Total Test Reviews: 1715


# **Data Preprocessing**

In [7]:
def clean_text(text):
    new_text = []
    text.lower()
    for idx in range(0, len(text)):
        if 2432 <= ord(text[idx]) <= 2559 or ord(text[idx])== 32 or ord('a') <= ord(text[idx]) <= ord('z'):
            new_text.append(text[idx])
    return new_text

In [ ]:
train_reviews = train_data['Reviews'].values.tolist()
train_reviews

In [ ]:
test_reviews = test_data['Reviews'].values.tolist()
test_reviews

In [10]:
pip install BnPreprocessing

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


In [11]:
import BnPreprocessing as pp

In [12]:
train_data['Reviews'] = train_data['Reviews'].apply(lambda x: pp.remove_punc(str(x)))
train_data['Reviews'] = train_data['Reviews'].apply(lambda x: pp.remove_digits(str(x)))
train_data['Reviews'] = train_data['Reviews'].apply(lambda x: pp.remove_emoticons(str(x)))
train_data['Reviews'] = train_data['Reviews'].apply(lambda x: pp.remove_sw(str(x)))

In [ ]:
train_reviews = train_data['Reviews'].values.tolist()
train_reviews

In [14]:
len(train_reviews)

6859

In [15]:
import re
for idx in range(0, len(train_reviews)):
    train_reviews[idx] = train_reviews[idx].lower()
    train_reviews[idx] = "".join(i for i in train_reviews[idx] if 2432 <= ord(i) <= 2559 or ord('a') <= ord(i) <= ord('z') or ord(i)== 32)
    train_reviews[idx] = re.sub(' +', ' ', train_reviews[idx])

In [ ]:
train_reviews

In [17]:
test_data['Reviews'] = test_data['Reviews'].apply(lambda x: pp.remove_punc(str(x)))
test_data['Reviews'] = test_data['Reviews'].apply(lambda x: pp.remove_digits(str(x)))
test_data['Reviews'] = test_data['Reviews'].apply(lambda x: pp.remove_emoticons(str(x)))
test_data['Reviews'] = test_data['Reviews'].apply(lambda x: pp.remove_sw(str(x)))

In [ ]:
test_reviews = test_data['Reviews'].values.tolist()
test_reviews

In [ ]:
for idx in range(0, len(test_reviews)):
    test_reviews[idx] = test_reviews[idx].lower()
    test_reviews[idx] = "".join(i for i in test_reviews[idx] if 2432 <= ord(i) <= 2559 or ord('a') <= ord(i) <= ord('z') or ord(i)== 32)
    test_reviews[idx] = re.sub(' +', ' ', test_reviews[idx])
test_reviews

In [20]:
reviews = train_reviews + test_reviews
len(reviews)

8574

# **Tokenization**

In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)
train_sequences = tokenizer.texts_to_sequences(train_reviews)
test_sequences = tokenizer.texts_to_sequences(test_reviews)

In [22]:
max_len = 0
for sequence in train_sequences:
  max_len = max(max_len, len(sequence))
max_len

422

In [23]:
max_len = 0
for sequence in test_sequences:
  max_len = max(max_len, len(sequence))
max_len

397

In [24]:
max_len = 450

In [25]:
x = pad_sequences(train_sequences, maxlen = max_len)
x_test = pad_sequences(test_sequences, maxlen = max_len)

In [26]:
word_index = tokenizer.word_index
print(len(word_index))

19521


In [27]:
y = train_data['Sentiment'].to_numpy()

# **Train-validation Split**

In [28]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, random_state = 10)

In [29]:
y_train_index, y_train_cnt = np.unique(y_train, return_counts = True)
for idx in range(y_train_index.shape[0]):
  print(y_train_index[idx], ':', y_train_cnt[idx])

0 : 2316
1 : 3171


In [30]:
y_val_index, y_val_cnt = np.unique(y_val, return_counts = True)
for idx in range(y_val_index.shape[0]):
  print(y_val_index[idx], ':', y_val_cnt[idx])

0 : 554
1 : 818


In [31]:
max_words = 20000

# **DL Model**

In [49]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [33]:
from keras import backend as K
K.clear_session()

In [34]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [50]:
model = Sequential()
model.add(Embedding(max_words, 128, input_length = max_len))
model.add(Bidirectional(LSTM(10, dropout = 0.3)))
model.add(Dense(3, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = [f1_m, precision_m, recall_m]
)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 450, 128)          2560000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 20)               11120     
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 3)                 63        
                                                                 
 dense_4 (Dense)             (None, 1)                 4         
                                                                 
Total params: 2,571,187
Trainable params: 2,571,187
Non-trainable params: 0
_________________________________________________________________


In [51]:
history = model.fit(x_train, y_train, batch_size = 64, epochs = 5, validation_data = (x_val, y_val))

Epoch 1/5
86/86 [==============================] - 34s 352ms/step - loss: 0.6139 - f1_m: 0.7802 - precision_m: 0.7550 - recall_m: 0.8377 - val_loss: 0.4373 - val_f1_m: 0.8888 - val_precision_m: 0.8649 - val_recall_m: 0.9170
Epoch 2/5
86/86 [==============================] - 28s 325ms/step - loss: 0.3276 - f1_m: 0.9129 - precision_m: 0.8982 - recall_m: 0.9316 - val_loss: 0.2984 - val_f1_m: 0.9046 - val_precision_m: 0.9016 - val_recall_m: 0.9091
Epoch 3/5
86/86 [==============================] - 29s 332ms/step - loss: 0.1988 - f1_m: 0.9488 - precision_m: 0.9415 - recall_m: 0.9582 - val_loss: 0.2850 - val_f1_m: 0.9099 - val_precision_m: 0.9105 - val_recall_m: 0.9113
Epoch 4/5
86/86 [==============================] - 28s 328ms/step - loss: 0.1335 - f1_m: 0.9676 - precision_m: 0.9627 - recall_m: 0.9734 - val_loss: 0.3102 - val_f1_m: 0.8990 - val_precision_m: 0.9199 - val_recall_m: 0.8816
Epoch 5/5
86/86 [==============================] - 28s 321ms/step - loss: 0.1028 - f1_m: 0.9754 - precis

In [52]:
y_train_pred = model.predict(x_train, batch_size = 64)
y_train_pred_labels = np.where(y_train_pred > 0.5, 1, 0)
print('F1 Score:', f1_score(y_train, y_train_pred_labels, average = 'micro'))
print(classification_report(y_train, y_train_pred_labels))

86/86 [==============================] - 6s 56ms/step
F1 Score: 0.9828685985055586
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      2316
           1       0.98      0.99      0.99      3171

    accuracy                           0.98      5487
   macro avg       0.98      0.98      0.98      5487
weighted avg       0.98      0.98      0.98      5487



In [53]:
y_val_pred = model.predict(x_val, batch_size = 64)
y_val_pred_labels = np.where(y_val_pred > 0.5, 1, 0)
print('F1 Score:', f1_score(y_val, y_val_pred_labels, average = 'micro'))
print(classification_report(y_val, y_val_pred_labels))

22/22 [==============================] - 1s 50ms/step
F1 Score: 0.8972303206997084
              precision    recall  f1-score   support

           0       0.87      0.88      0.87       554
           1       0.92      0.91      0.91       818

    accuracy                           0.90      1372
   macro avg       0.89      0.89      0.89      1372
weighted avg       0.90      0.90      0.90      1372



In [54]:
y_test_pred = model.predict(x_test, batch_size = 64)
y_test_pred_labels = np.where(y_test_pred > 0.5, 1, 0)

27/27 [==============================] - 1s 55ms/step


In [ ]:
y_test_pred_labels = y_test_pred_labels.tolist()
y_test_pred_labels

In [56]:
output_data = []
for idx in range(0, len(y_test_pred_labels)):
    output_data.append([idx, y_test_pred_labels[idx][0]])
output_df = pd.DataFrame(output_data, columns = ['Id', 'Sentiment'])
output_df

,Id,Sentiment
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
1710,1710,1
1711,1711,0
1712,1712,0
1713,1713,0


In [57]:
output_df.to_csv('prediction.csv', index = False)